In [2]:
!pip install langchain-community faiss-cpu openai tiktoken langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 56.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.7/306.7 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests

In [1]:
!pip install git+https://github.com/openai/whisper.git
!pip install ffmpeg-python

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-f66pzff4
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-f66pzff4
  Resolved https://github.com/openai/whisper.git to commit 271445b2f24f00f8175c4fb7ae91876f7451dfc1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 7.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.2 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803557 sha256=5294eadd424e8cc217b2acafde828789f6ab0a509c9093bfe6f69fdeb1b1211e
  Stored in directory: /tmp/pip-ephem-wheel-cache-9dy_v_n6/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [3]:
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores.faiss import FAISS
from langchain.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings
import faiss

In [4]:
api_key=''

In [5]:
user_data = {
  "user_1": [
    {
      "event_id": "1",
      "subject": "Team Standup Meeting",
      "start_time": "2024-11-10 11:00",
      "description": "Daily team sync-up to discuss project progress and blockers."
    },
    {
      "event_id": "2",
      "subject": "Client Presentation",
      "start_time": "2024-11-11 13:00",
      "description": "Present project deliverables to the client."
    },
    {
      "event_id": "3",
      "subject": "Morning Yoga Session",
      "start_time": "2024-11-11 09:00",
      "description": "A relaxing start to the day with a 1-hour yoga session."
    },
    {
      "event_id": "4",
      "subject": "Project Kickoff Meeting",
      "start_time": "2024-11-12 11:00",
      "description": "Initial meeting to align stakeholders on project goals and timelines."
    },
    {
      "event_id": "5",
      "subject": "Lunch with Mentor",
      "start_time": "2024-11-12 13:00",
      "description": "Catch up and discuss career growth strategies over lunch."
    },
    {
      "event_id": "6",
      "subject": "Code Review Session",
      "start_time": "2024-11-13 09:00",
      "description": "Review and provide feedback on recent pull requests."
    },
    {
      "event_id": "7",
      "subject": "Quarterly Planning",
      "start_time": "2024-11-13 11:00",
      "description": "Plan the next quarter's objectives with the team."
    },
    {
      "event_id": "8",
      "subject": "1:1 with Manager",
      "start_time": "2024-11-14 13:00",
      "description": "Discuss performance and goals in a one-on-one meeting."
    },
    {
      "event_id": "9",
      "subject": "Technical Workshop",
      "start_time": "2024-11-14 09:00",
      "description": "Attend a workshop on advanced cloud architecture."
    },
    {
      "event_id": "10",
      "subject": "Weekly Retrospective",
      "start_time": "2024-11-15 11:00",
      "description": "Reflect on the week's progress and identify areas for improvement."
    },
    {
      "event_id": "30",
      "subject": "Hackathon Kickoff",
      "start_time": "2024-11-25 09:00",
      "description": "Start of the annual company hackathon."
    }
  ],
  "user_2": [
    {
      "event_id": "1",
      "subject": "Marketing Strategy Meeting",
      "start_time": "2024-11-10 11:00",
      "description": "Discuss Q1 marketing strategies with the team."
    },
    {
      "event_id": "2",
      "subject": "Product Launch Event",
      "start_time": "2024-11-11 13:00",
      "description": "Launch the new product line to the public."
    },
    {
      "event_id": "3",
      "subject": "Morning Run",
      "start_time": "2024-11-11 09:00",
      "description": "Start the day with a refreshing 5K run."
    },
    {
      "event_id": "4",
      "subject": "Budget Review Meeting",
      "start_time": "2024-11-12 11:00",
      "description": "Review the department's budget for the next quarter."
    },
    {
      "event_id": "5",
      "subject": "Client Networking Lunch",
      "start_time": "2024-11-12 13:00",
      "description": "Build relationships with key clients over lunch."
    },
    {
      "event_id": "6",
      "subject": "Content Planning Session",
      "start_time": "2024-11-13 09:00",
      "description": "Plan content for the upcoming social media campaign."
    },
    {
      "event_id": "7",
      "subject": "Sales Team Training",
      "start_time": "2024-11-13 11:00",
      "description": "Training session for the sales team on new CRM software."
    },
    {
      "event_id": "8",
      "subject": "Vendor Call",
      "start_time": "2024-11-14 13:00",
      "description": "Discuss contract terms with a new vendor."
    },
    {
      "event_id": "9",
      "subject": "Leadership Workshop",
      "start_time": "2024-11-14 09:00",
      "description": "Participate in a workshop on leadership skills."
    },
    {
      "event_id": "10",
      "subject": "Team Building Activity",
      "start_time": "2024-11-15 11:00",
      "description": "Engage in fun activities to strengthen team bonds."
    },
    {
      "event_id": "30",
      "subject": "Year-End Review",
      "start_time": "2024-11-25 09:00",
      "description": "Comprehensive review of the year's performance and achievements."
    }
  ]
}


In [6]:
def setup_user_data(user_id, events):
    documents = [
        Document(page_content=f"{event['subject']} on {event['start_time']} - {event['description']}", metadata=event)
        for event in events
    ]

    embedding_model = OpenAIEmbeddings(model="text-embedding-3-small", api_key=api_key)
    embeddings = embedding_model.embed_documents([doc.page_content for doc in documents])
    

    dimension = len(embeddings[0])
    index = faiss.IndexFlatL2(dimension)
    docstore = InMemoryDocstore({str(i): doc for i, doc in enumerate(documents)})
    index_to_docstore_id = {i: str(i) for i in range(len(documents))}
    
    faiss_index = FAISS(
        index=index,
        docstore=docstore,
        index_to_docstore_id=index_to_docstore_id,
        embedding_function=embedding_model.embed_query
    )
    faiss_index.add_texts([doc.page_content for doc in documents])
    
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    
    return faiss_index, memory

In [7]:
user_chains = {}
for user_id, events in user_data.items():
    faiss_index, memory = setup_user_data(user_id, events)
    retriever = faiss_index.as_retriever(search_kwargs={"k": 3})
    prompt_template = PromptTemplate.from_template(template="""
        You are a helpful assistant providing information about events.

        Context: {context}

        Chat History: {chat_history}

        Question: {question}

        Answer:
    """)
    llm = ChatOpenAI(temperature=0.3, model="gpt-4o-mini", openai_api_key=api_key)
    
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        return_source_documents=False,
        combine_docs_chain_kwargs={"prompt": prompt_template}
    )
    user_chains[user_id] = qa_chain

/tmp/ipykernel_30/3808713830.py:24: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [8]:

def query_user(user_id, question):
    qa_chain = user_chains[user_id]
    response = qa_chain.invoke({"question": question, "chat_history": []})
    return response["answer"]

# Test queries
print("User 1 Query:")
print(query_user("user_1", "What events do I have this week?"))


User 1 Query:
This week, you have the following events scheduled:

1. **Team Standup Meeting** on **2024-11-10 at 11:00** - A daily team sync-up to discuss project progress and blockers.
   
2. **Client Presentation** on **2024-11-11 at 13:00** - Present project deliverables to the client.

3. **Weekly Retrospective** on **2024-11-15 at 11:00** - Reflect on the week's progress and identify areas for improvement.


In [11]:
import whisper
import tempfile
import os
import ffmpeg
from IPython.display import Audio

def transcribe_audio_kaggle(audio_file_path, model_name="base"):
    model = whisper.load_model(model_name)
    result = model.transcribe(audio_file_path)
    return result['text'] 

def query_user_with_audio(user_id, audio_file_path):
    transcription = transcribe_audio_kaggle(audio_file_path)
    print(f"Transcribed Question: {transcription}")
    
    answer = query_user(user_id, transcription)
    return answer

audio_path = '/kaggle/input/audioo/Rev.mp3'
print(query_user_with_audio("user_1", audio_path))

/opt/conda/lib/python3.10/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
/opt/conda/li

Transcribed Question:  Do I have a meeting tomorrow?
Yes, you have a **Team Standup Meeting** scheduled for tomorrow, **2024-11-10 at 11:00**.
